# 4단계 YOLOv8 전이학습

## 4.1 YOLO에 입력 가능한 형태로 라벨 데이터 통합 및 변환
temp_label에 저장된 자세 정보를 읽어 원본 labels의 클래스 번호를 갱신한 뒤 maple-vision-search/data/processed/yolo_label에 저장

In [1]:
import os
import re
from tqdm import tqdm

# 1. 경로 설정
PROJECT_ROOT = os.path.expanduser("~/maple-vision-search")
RAW_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/raw/all_generated/labels")
TEMP_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/temp_label")
YOLO_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/yolo_label")

if not os.path.exists(YOLO_LABEL_DIR):
    os.makedirs(YOLO_LABEL_DIR)

# 2. 데이터 매핑 데이터 구조 생성
# { '원본파일명': { 객체인덱스: 포즈번호 } }
mapping_table = {}

print("단계 1: 오토 라벨링 결과(Pose) 파싱 중...")
temp_files = [f for f in os.listdir(TEMP_LABEL_DIR) if f.endswith('.txt')]

for f in temp_files:
    # 파일명 규칙: '이미지파일명_객체인덱스.txt'라고 가정 (예: img_0_0.txt)
    # 2단계 character_crop.ipynb에서 저장한 규칙에 맞게 정규표현식 수정 필요
    match = re.match(r"(.+)_(\d+)\.txt", f)
    if match:
        original_img_name = match.group(1)
        obj_idx = int(match.group(2))
        
        with open(os.path.join(TEMP_LABEL_DIR, f), 'r') as file:
            pose_id = file.read().strip()
            
        if original_img_name not in mapping_table:
            mapping_table[original_img_name] = {}
        mapping_table[original_img_name][obj_idx] = pose_id

# 3. 원본 좌표와 결합하여 최종 YOLO 라벨 생성
print("단계 2: 원본 좌표와 결합 및 최종 라벨 생성 중...")
for img_name, objects in tqdm(mapping_table.items()):
    raw_label_path = os.path.join(RAW_LABEL_DIR, f"{img_name}.txt")
    
    if not os.path.exists(raw_label_path):
        continue
        
    with open(raw_label_path, 'r') as f:
        lines = f.readlines()
        
    updated_lines = []
    for idx, line in enumerate(lines):
        parts = line.strip().split()
        if not parts: continue
        
        # 해당 객체(인덱스)의 포즈 번호가 있다면 교체, 없으면 기존값 유지
        new_pose = objects.get(idx, parts[0])
        
        # YOLO 포맷: [pose_id] [x_center] [y_center] [width] [height]
        new_line = f"{new_pose} {' '.join(parts[1:])}\n"
        updated_lines.append(new_line)
        
    # 결과 저장
    with open(os.path.join(YOLO_LABEL_DIR, f"{img_name}.txt"), 'w') as f:
        f.writelines(updated_lines)

print(f"최종 완료: {len(mapping_table)}개의 통합 라벨 파일이 '{YOLO_LABEL_DIR}'에 저장되었습니다.")

단계 1: 오토 라벨링 결과(Pose) 파싱 중...
단계 2: 원본 좌표와 결합 및 최종 라벨 생성 중...


100%|██████████| 489/489 [00:00<00:00, 648849.94it/s]

최종 완료: 489개의 통합 라벨 파일이 '/home/ssy/maple-vision-search/data/processed/yolo_label'에 저장되었습니다.


In [2]:
import os
import re
from tqdm import tqdm

# 1. 경로 설정 (서영님의 구조 재확인)
PROJECT_ROOT = os.path.expanduser("~/maple-vision-search")
RAW_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/raw/all_generated/labels")
TEMP_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/temp_label")
YOLO_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/yolo_label")

if not os.path.exists(YOLO_LABEL_DIR):
    os.makedirs(YOLO_LABEL_DIR)

# 2. 데이터 매핑 데이터 구조 생성
mapping_table = {}

print("--- 단계 1: 오토 라벨링 결과(Pose) 파싱 시작 ---")
temp_files = [f for f in os.listdir(TEMP_LABEL_DIR) if f.endswith('.txt')]

if not temp_files:
    print(f"오류: {TEMP_LABEL_DIR} 에 txt 파일이 없습니다.")
else:
    print(f"발견된 라벨 파일 수: {len(temp_files)}개")
    # 샘플 파일명 출력으로 정규식 확인
    print(f"파일명 샘플: {temp_files[0]}")

for f in temp_files:
    # 파일명 예시: 'maple_0.png_0.txt' 또는 'maple_0_0.txt' 대응을 위해 정규식 수정
    # 확장자가 포함되어 있을 가능성을 고려하여 (.+?) 사용
    match = re.match(r"(.+)_(\d+)\.txt", f)
    if match:
        original_img_name = match.group(1)
        # 만약 original_img_name에 .png 등이 포함되어 있다면 제거
        original_img_name = os.path.splitext(original_img_name)[0]
        
        obj_idx = int(match.group(2))
        
        with open(os.path.join(TEMP_LABEL_DIR, f), 'r') as file:
            pose_id = file.read().strip()
            
        if original_img_name not in mapping_table:
            mapping_table[original_img_name] = {}
        mapping_table[original_img_name][obj_idx] = pose_id
    else:
        # 정규식 매칭 실패 시 출력
        print(f"정규식 매칭 실패 파일: {f}")

print(f"매핑된 원본 이미지 수: {len(mapping_table)}개")

# 3. 원본 좌표와 결합하여 최종 YOLO 라벨 생성
print("\n--- 단계 2: 원본 좌표 결합 및 저장 시작 ---")
if not mapping_table:
    print("오류: mapping_table이 비어있어 다음 단계를 진행할 수 없습니다. 파일명을 확인하세요.")
else:
    for img_name, objects in tqdm(mapping_table.items()):
        raw_label_path = os.path.join(RAW_LABEL_DIR, f"{img_name}.txt")
        
        if not os.path.exists(raw_label_path):
            # 경로가 없는 경우 출력 (파일명 불일치 확인용)
            # print(f"원본 라벨 없음: {raw_label_path}")
            continue
            
        with open(raw_label_path, 'r') as f:
            lines = f.readlines()
            
        updated_lines = []
        for idx, line in enumerate(lines):
            parts = line.strip().split()
            if not parts: continue
            
            # 해당 객체의 포즈 번호로 교체
            new_pose = objects.get(idx, parts[0])
            new_line = f"{new_pose} {' '.join(parts[1:])}\n"
            updated_lines.append(new_line)
            
        with open(os.path.join(YOLO_LABEL_DIR, f"{img_name}.txt"), 'w') as f:
            f.writelines(updated_lines)

print(f"\n최종 완료: '{YOLO_LABEL_DIR}' 폴더 내 파일 목록: {os.listdir(YOLO_LABEL_DIR)[:5]}... (총 {len(os.listdir(YOLO_LABEL_DIR))}개)")

--- 단계 1: 오토 라벨링 결과(Pose) 파싱 시작 ---
발견된 라벨 파일 수: 1467개
파일명 샘플: d8caa21f-6238-4182-ad9c-45968af6b1c6_0.txt
매핑된 원본 이미지 수: 489개

--- 단계 2: 원본 좌표 결합 및 저장 시작 ---


100%|██████████| 489/489 [00:00<00:00, 455578.56it/s]


최종 완료: '/home/ssy/maple-vision-search/data/processed/yolo_label' 폴더 내 파일 목록: []... (총 0개)


In [4]:
import os
import re
import shutil
import yaml
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. 경로 설정
PROJECT_ROOT = os.path.expanduser("~/maple-vision-search")
RAW_IMG_DIR = os.path.join(PROJECT_ROOT, "data/raw/images")
RAW_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/raw/labels")
TEMP_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/temp_label")
YOLO_LABEL_DIR = os.path.join(PROJECT_ROOT, "data/processed/yolo_label")

# 학습용 최종 데이터셋 경로
YOLO_DATASET_ROOT = os.path.join(PROJECT_ROOT, "data/yolo_dataset")

for path in [YOLO_LABEL_DIR, YOLO_DATASET_ROOT]:
    os.makedirs(path, exist_ok=True)

# ---------------------------------------------------------
# STEP 1: 라벨 통합 (yolo_label 생성)
# ---------------------------------------------------------
mapping_table = {}
temp_files = [f for f in os.listdir(TEMP_LABEL_DIR) if f.endswith('.txt')]

for f in temp_files:
    # 정규식 수정: 파일명에 .png나 .jpg가 포함된 경우도 대응
    # 예: 'maple_0.png_0.txt' -> group(1): maple_0.png, group(2): 0
    match = re.search(r"(.+)_(\d+)\.txt$", f)
    if match:
        full_name = match.group(1)
        # 확장자 제거 (maple_0.png -> maple_0)
        img_basename = os.path.splitext(full_name)[0]
        obj_idx = int(match.group(2))
        
        with open(os.path.join(TEMP_LABEL_DIR, f), 'r') as file:
            pose_id = file.read().strip()
            
        if img_basename not in mapping_table:
            mapping_table[img_basename] = {}
        mapping_table[img_basename][obj_idx] = pose_id

print(f"매핑된 원본 이미지: {len(mapping_table)}개")

# 결합 실행
for img_name, objects in mapping_table.items():
    raw_path = os.path.join(RAW_LABEL_DIR, f"{img_name}.txt")
    if not os.path.exists(raw_path): continue
    
    with open(raw_path, 'r') as f:
        lines = f.readlines()
    
    final_lines = []
    for idx, line in enumerate(lines):
        parts = line.strip().split()
        if not parts: continue
        # 오토 라벨링된 포즈로 교체 (매핑 없으면 기존값 유지)
        parts[0] = objects.get(idx, parts[0])
        final_lines.append(" ".join(parts) + "\n")
    
    with open(os.path.join(YOLO_LABEL_DIR, f"{img_name}.txt"), 'w') as f:
        f.writelines(final_lines)

# ---------------------------------------------------------
# STEP 2: 데이터셋 분할 (Train/Val Split)
# ---------------------------------------------------------
images = [f for f in os.listdir(RAW_IMG_DIR) if f.endswith(('.png', '.jpg'))]
# 라벨이 존재하는 이미지들만 필터링
valid_images = [img for img in images if os.path.exists(os.path.join(YOLO_LABEL_DIR, os.path.splitext(img)[0] + '.txt'))]

train_imgs, val_imgs = train_test_split(valid_images, test_size=0.2, random_state=42)

def move_files(file_list, split):
    img_dest = os.makedirs(os.path.join(YOLO_DATASET_ROOT, f"images/{split}"), exist_ok=True)
    lbl_dest = os.makedirs(os.path.join(YOLO_DATASET_ROOT, f"labels/{split}"), exist_ok=True)
    
    for img_name in file_list:
        base_name = os.path.splitext(img_name)[0]
        # 이미지 복사
        shutil.copy(os.path.join(RAW_IMG_DIR, img_name), 
                    os.path.join(YOLO_DATASET_ROOT, f"images/{split}/{img_name}"))
        # 라벨 복사
        shutil.copy(os.path.join(YOLO_LABEL_DIR, f"{base_name}.txt"), 
                    os.path.join(YOLO_DATASET_ROOT, f"labels/{split}/{base_name}.txt"))

move_files(train_imgs, 'train')
move_files(val_imgs, 'val')

# ---------------------------------------------------------
# STEP 3: data.yaml 생성
# ---------------------------------------------------------
data_config = {
    'path': YOLO_DATASET_ROOT,
    'train': 'images/train',
    'val': 'images/val',
    'nc': 10,
    'names': ['stand', 'walk', 'jump', 'prone', 'swing', 'stab', 'alert', 'fly', 'ladder', 'rope']
}

with open(os.path.join(YOLO_DATASET_ROOT, 'data.yaml'), 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print(f"데이터셋 준비 완료: {len(train_imgs)} train / {len(val_imgs)} val")

매핑된 원본 이미지: 489개
데이터셋 준비 완료: 391 train / 98 val
